In [1]:
from functions import *
from settings import *
from pipeline import *

%store -r __RequiredPackages
%store -r __JupyterOptions

In [10]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [2]:
__RequiredPackages

In [3]:
__JupyterOptions

Populating the interactive namespace from numpy and matplotlib


In [4]:
# Import data
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
# Initiate prediction df
preds_df = pd.DataFrame({'Id': test['Id'], 'SalePrice': np.repeat(0, len(test['Id']))})

In [6]:
# Standard preprocessing
data = standard_preprocessing_function(data, NA_means_not_there_cols, scale_cont_cols)
test = standard_preprocessing_function(test, NA_means_not_there_cols, scale_cont_cols, train=False)

In [8]:
# Shuffle rows
#data = data.sample(frac=1)

# Seperate target and predictos
X = data.drop(target, axis=1)
y = data[target]

# Transform predictors
prepper = leakage_preventive_preprocessing_function(target, cont_impute_cols, cat_impute_cols, mean_enc_cols, 
                                                    square_features, keep_features)
X = prepper.fit_transform(X, y)
test = prepper.transform(test)

In [ ]:
# # Seperate target and predictos
# X = data.drop(target, axis=1)
# y = data[target]

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# # Transform predictors
# prepper = leakage_preventive_preprocessing_function(target, cont_impute_cols, cat_impute_cols, mean_enc_cols, 
#                                                     square_features, keep_features)
# X_train = prepper.fit_transform(X_train, y_train)
# X_test = prepper.transform(X_test)

# GBRmodel = GradientBoostingRegressor(learning_rate=0.075,
#                                   max_depth=4,
#                                   min_samples_split=16,
#                                   max_features=5,
#                                   subsample=0.75,
#                                   n_estimators=200,
#                                   random_state=dev_seed)

# ridgemodel = Ridge(alpha=4, random_state=dev_seed)

# RFmodel = RandomForestRegressor(n_estimators=100, max_depth=14, min_samples_split=4, max_features=5,
#                                 max_samples=0.99, random_state=dev_seed, n_jobs=n_jobs)

# SVmodel = SVR(kernel = 'linear', tol = 0.0000001, C = 0.1, epsilon = 0.1)

# GBRmodel = GBRmodel.fit(X_train[keep_features], y_train)
# GBRpreds = GBRmodel.predict(X_test[keep_features])

# ridgemodel = ridgemodel.fit(X_train, y_train)
# ridgepreds = ridgemodel.predict(X_test)

# # RFmodel = RFmodel.fit(X_train[keep_features], y_train)
# # RFpreds = RFmodel.predict(X_test[keep_features])

# SVmodel = SVmodel.fit(X_train, y_train)
# SVpreds = SVmodel.predict(X_test)

# final_preds = np.sum([0.15*ridgepreds, 0.6*GBRpreds, 0.25*SVpreds], axis=0)

# np.sqrt(mean_squared_error(y_test, final_preds))

In [ ]:
sns.scatterplot(y_test, preds)

In [ ]:
sns.scatterplot(y_test, preds-y_test)

In [ ]:
# Fit and predict
model = GradientBoostingRegressor(learning_rate=0.075,
                                  max_depth=4,
                                  min_samples_split=16,
                                  max_features=5,
                                  subsample=0.75,
                                  n_estimators=200,
                                  random_state=dev_seed)

model = model.fit(X, y)
preds_df['SalePrice'] = np.exp(model.predict(test))

In [11]:
GBRmodel = GradientBoostingRegressor(learning_rate=0.075,
                                  max_depth=4,
                                  min_samples_split=16,
                                  max_features=5,
                                  subsample=0.75,
                                  n_estimators=200,
                                  random_state=dev_seed)

ridgemodel = Ridge(alpha=4, random_state=dev_seed)

RFmodel = RandomForestRegressor(n_estimators=100, max_depth=14, min_samples_split=4, max_features=5,
                                max_samples=0.99, random_state=dev_seed, n_jobs=n_jobs)

SVmodel = SVR(kernel = 'linear', tol = 0.0000001, C = 0.1, epsilon = 0.1)

GBRmodel = GBRmodel.fit(X[keep_features], y)
GBRpreds = GBRmodel.predict(test[keep_features])

ridgemodel = ridgemodel.fit(X, y)
ridgepreds = ridgemodel.predict(test)

SVmodel = SVmodel.fit(X, y)
SVpreds = SVmodel.predict(test)

final_preds = np.sum([0.15*ridgepreds, 0.6*GBRpreds, 0.25*SVpreds], axis=0)
preds_df['SalePrice'] = np.exp(final_preds)

In [12]:
preds_df.shape

(1459, 2)

In [13]:
preds_df.head()

,Id,SalePrice
0,1461,113918.871349
1,1462,157148.286596
2,1463,184221.401300
3,1464,195339.051532
4,1465,191606.478793


In [14]:
preds_df.to_csv('output/preds_20200703_1.csv', index=False)

In [ ]:
preds_df.head()

In [ ]:
sns.distplot(preds_df['SalePrice'])
sns.distplot(data['SalePrice'], color='red')

In [15]:
preds_df[['SalePrice']].describe(), data[['SalePrice']].describe()

(           SalePrice
 count  1459.000000  
 mean   178988.361136
 std    78967.073209 
 min    40632.636959 
 25%    127667.186082
 50%    156953.418610
 75%    209538.053982
 max    784944.405764,
            SalePrice
 count  1458.000000  
 mean   180932.919067
 std    79495.055285 
 min    34900.000000 
 25%    129925.000000
 50%    163000.000000
 75%    214000.000000
 max    755000.000000)